I didn't manage to train this model - might be because I ended up using too a small batch size, maybe I gave up too quickly, or maybe for some other reason.

Leaving this here as a reference - it modifies the base architecture so that it can be run with any input size and uses forward hook to grab cnn activations.

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = pretrainedmodels.se_resnext50_32x4d()
        self.cnn.avg_pool = Lambda(lambda x: x.view(BS, 2048, -1).mean(-1)) # this allows the model to be run on any size of input
        self.cnn_out = []
        nn.Module.register_forward_hook(self.cnn.layer4, self.grab_output)
        
        self.head = create_head(4096, 5004, [2048])
        self.ada_concat = AdaptiveConcatPool2d(1)
            
    def grab_output(self, mod, inp, out):
        self.cnn_out.append(out)
        
    def forward(self, ims_a, ims_b):
        self.cnn(ims_a)
        cnn_out_a = self.cnn_out[0]
        out_a = self.head(cnn_out_a)
        self.cnn_out = []
        
        self.cnn(ims_b)
        cnn_out_b = self.cnn_out[0]
        out_b = self.head(cnn_out_b)
        self.cnn_out = []
        
        return out_a, out_b, self.ada_concat(cnn_out_a).squeeze(), self.ada_concat(cnn_out_b).squeeze()